In [4]:
from datasets import load_dataset
DATASET_NAME = 'race'
dataset = load_dataset("ehovy/race", "all", split=["train", "test"])

In [5]:
def get_prompt(dataset, index, has_choice=False):
    # try:
        json_line = dataset[index]
        question = json_line["question"]
        choice_texts = [x.replace("\n", " ").replace(' ( ) ','') for x in json_line["options"]]
        candidates = " ".join(
            [
                f"( ) {text}"
                for text in choice_texts
            ]
        )
        answer_key = json_line["answer"][0]
        answer_key_idx = ord(answer_key) - \
            (ord("A") if answer_key in 'ABCDE' else ord("1"))
        answer_text = choice_texts[answer_key_idx]
        fact = f"{json_line['article']}{'' if json_line['article'][-1]=='.' else '.'}"
        prompt = f"{fact} {question} \\n {candidates}"
        return prompt, answer_text

In [6]:
from tqdm import tqdm
import pickle

for index, dataname in enumerate(['train', 'test']):
    container = []
    for ques_index, ques in enumerate(tqdm(dataset[index])):
        container.append(
            get_prompt(dataset[index], ques_index, has_choice=False)
        )
    pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 4934/4934 [00:00<00:00, 10355.36it/s]
